<a href="https://colab.research.google.com/github/KeoniM/NFL_Data_Cleaning/blob/main/NFL_Plays_Week2_2023_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PURPOSE:**
- Accurately clean a week's worth of play data
  - Season 2023 -> Week 2

**NOTE:**
- What makes version 2 different than version 1 is the data being used. Although the core of the data is identical to the original, NFL.com has updated their formatting of how they display their data which has been scraped and used here. So minor adjustments will have to be made in creating the new version but I also see a beautiful opportunity to clean the older version here. Make the code more readible, organized and efficient.

# MOUNTING AND IMPORTS

In [2]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Used to access personal google cloud services
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
# Imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Database access
from google.cloud import bigquery

# LOADING DATA (BigQuery)

In [5]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

## Season 2023 Week 2

In [6]:
# Grabbing all plays from 2023 Week 2 NFL Sesason
nfl_plays_week2_2023_query = """
                             SELECT *
                             FROM `nfl-data-430702.NFL_Scores_v2.NFL-Plays-Week2_2023`
                             """

# Running psuedo query, and returns the amount of bytes it will take to run query
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_plays_week2_2023_query, job_config=dry_run_config)
print("This query will process {} gigabytes.".format(dry_run_query.total_bytes_processed/10**9))

# Running query (Being mindful of the amount of data being grabbed)
# Will grab a maximum of a Gigabyte
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_plays_week2_2023_query, job_config=safe_config)

This query will process 0.00063942 gigabytes.


In [7]:
# Putting data attained from query into a dataframe
week2_2023_plays = safe_config_query.to_dataframe()

In [8]:
week2_2023_plays.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayStart,PlayTimeFormation,PlayDescription
0,2023,Week 1,THU,09/07,Lions,Chiefs,1st Quarter,1,Detroit Lions,0,1,0,Kickoff from KC 35,None,Kickoff,— H.Butker kicks 65 yards from KC 35 to end zo...
1,2023,Week 1,THU,09/07,Lions,Chiefs,1st Quarter,1,Detroit Lions,0,2,0,7 Yard Run,1st & 10 at DET 25,15:00 1st,— D.Montgomery right guard to DET 32 for 7 yar...
2,2023,Week 1,THU,09/07,Lions,Chiefs,1st Quarter,1,Detroit Lions,0,3,0,Pass Incomplete,2nd & 3 at DET 32,14:29 1st,— J.Goff pass incomplete short right.
3,2023,Week 1,THU,09/07,Lions,Chiefs,1st Quarter,1,Detroit Lions,0,4,0,Pass Incomplete,3rd & 3 at DET 32,14:25 1st Shotgun,— J.Goff pass incomplete deep right to J.Reyno...
4,2023,Week 1,THU,09/07,Lions,Chiefs,1st Quarter,1,Detroit Lions,0,5,0,Punt,4th & 3 at DET 32,14:21 1st,"— J.Fox punts 61 yards to KC 7, Center-S.Daly...."


# CATEGORIZE PLAYS
- The goal here is to parse out the different values for 'PlayOutcome'
  - Here is where I will separate different types of plays
    - ( pass / run / kickoff / etc..)

In [9]:
# All play outcomes from the game
# - From here we can categorize and clean plays accordingly
week2_2023_plays['PlayOutcome'].unique()

array(['Kickoff from KC 35', '7 Yard Run', 'Pass Incomplete', 'Punt',
       '5 Yard Run', '3 Yard Run', '10 Yard Pass', '5 Yard Pass',
       '2 Yard Run', '1 Yard Run', '13 Yard Pass', '21 Yard Pass',
       '4 Yard Run', '17 Yard Run', '6 Yard Run', '0 Yard Run',
       'Touchdown', 'Extra Point', 'Kickoff from DET 35', '19 Yard Pass',
       '14 Yard Pass', '16 Yard Run', '1 Yard Pass', 'Timeout',
       '9 Yard Pass', '-1 Yard Run', '8 Yard Pass', '-10 Yard Run',
       'Fumble', '-3 Yard Sack', '25 Yard Pass', '3 Yard Pass',
       '-10 Yard Penalty', '34 Yard Pass', '26 Yard Pass', '4 Yard Pass',
       '2 Yard Pass', '7 Yard Pass', '6 Yard Pass', '18 Yard Pass',
       '12 Yard Pass', '9 Yard Run', '41 Yard Penalty', 'Field Goal',
       '-12 Yard Penalty', '20 Yard Pass', '24 Yard Pass', '-2 Yard Run',
       '-5 Yard Penalty', '33 Yard Pass', '-3 Yard Run', '18 Yard Run',
       'Kickoff from CAR 35', '-6 Yard Pass', '8 Yard Run',
       '-8 Yard Sack', 'Turnover on Downs', '

In [10]:
# NOTES:
# - Currently, I am eyeing all unique play outcomes to categorizing them.
#   - This type of approach is not flexable because a play outcome can
#     arise that has not been seen yet.
#     - There may be more play outcomes in the future when working on a full season,
#       let alone all seasons and future games

# Play Types with complete cleaning methods (As far as this sample size goes)

# ~ OFFENSE ~
df_2023_pass_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Pass')]
df_2023_run_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Run')]
# ~ DEFENSE ~
df_2023_interception_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Interception')]
df_2023_sack_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Sack')]
# ~ SPECIAL TEAMS ~
df_2023_punt_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Punt')]
df_2023_kickoff_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Kickoff')]
# ~ SCORING ~
df_2023_touchdown_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Touchdown')]
df_2023_extrapoint_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Extra Point')]
df_2023_fieldgoal_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Field Goal')]
# df_2023_2pt_conversion_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('2PT Conversion')]
df_2023_2pt_conversion_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Conversion')]
# ~ OTHER ~
df_2023_fumble_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Fumble')]
df_2023_penalty_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Penalty')]
df_2023_turnover_on_downs_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Turnover on Downs')]
df_2023_timeout_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Timeout')]

## SANITY CHECK (All Plays Accounted for)
  - Once all plays have been categorized, will compare the sum of all plays in each category to the size of the original dataframe of plays.
    - Goal is to make sure the number of plays is the same.

In [11]:
# Categorized plays

plays_list = [df_2023_pass_week2,         # Offense
              df_2023_run_week2,
              df_2023_interception_week2, # Defense
              df_2023_sack_week2,
              df_2023_punt_week2,         # Special Teams
              df_2023_kickoff_week2,
              df_2023_touchdown_week2,    # Scoring
              df_2023_extrapoint_week2,
              df_2023_fieldgoal_week2,
              df_2023_2pt_conversion_week2,
              df_2023_fumble_week2,       # Other
              df_2023_penalty_week2,
              df_2023_turnover_on_downs_week2,
              df_2023_timeout_week2]

num_plays_categorized = 0

for plays in plays_list:
  num_plays_categorized = num_plays_categorized + len(plays)

num_plays_categorized == len(week2_2023_plays)

True

# PIPELINE
- ORDER
  1. Team Dictionary
    - Used to map team names with their acronyms
  2. Regular expressions
    - Used to find common patterns within raw data
  3. Transforming Data
    - So far, only label encoding
  4. Cleaning methods
    - Unique cleaning methods for each play type
  5. Main pipeline method
    - Control flow of cleaning methods

## 1. TEAM DICTIONARY

In [12]:
# KEY: Team name
# VALUE: Acronym of team

dict_teams = {
    'Cardinals': 'ARI', 'Falcons': 'ATL', 'Ravens': 'BAL', 'Bills': 'BUF', 'Panthers': 'CAR', 'Bears': 'CHI',
    'Bengals': 'CIN', 'Browns': 'CLE', 'Cowboys': 'DAL', 'Broncos': 'DEN', 'Lions': 'DET', 'Packers': 'GB',
    'Texans': 'HOU', 'Colts': 'IND', 'Jaguars': 'JAX', 'Chiefs': 'KC', 'Raiders': 'LV', 'Chargers': 'LAC',
    'Rams': 'LAR', 'Dolphins': 'MIA', 'Vikings': 'MIN', 'Patriots': 'NE', 'Saints': 'NO', 'Giants': 'NYG',
    'Jets': 'NYJ', 'Eagles': 'PHI', 'Steelers': 'PIT', '49ers': 'SF', 'Seahawks': 'SEA', 'Buccaneers': 'TB',
    'Titans': 'TEN', 'Commanders': 'WAS'
}

In [13]:
# KEY: Full Team name
# VALUE: Acronym of team

dict_teams_2 = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL', 'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL', 'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX', 'Kansas City Chiefs': 'KC', 'Las Vegas Raiders': 'LV', 'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LAR', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN', 'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT', 'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS'
}

In [14]:
# KEY: Acronym of team
# VALUE: Team name

dict_teams_3 = {
    'ARI': 'Arizona Cardinals', 'ATL': 'Atlanta Falcons', 'BAL': 'Baltimore Ravens', 'BUF': 'Buffalo Bills', 'CAR': 'Carolina Panthers', 'CHI': 'Chicago Bears',
    'CIN': 'Cincinnati Bengals', 'CLE': 'Cleveland Browns', 'DAL': 'Dallas Cowboys', 'DEN': 'Denver Broncos', 'DET': 'Detroit Lions', 'GB': 'Green Bay Packers',
    'HOU': 'Houston Texans', 'IND': 'Indianapolis Colts', 'JAX': 'Jacksonville Jaguars', 'KC': 'Kansas City Chiefs', 'LV': 'Las Vegas Raiders', 'LAC': 'Los Angeles Chargers',
    'LAR': 'Los Angeles Rams', 'MIA': 'Miami Dolphins', 'MIN': 'Minnesota Vikings', 'NE': 'New England Patriots', 'NO': 'New Orleans Saints', 'NYG': 'New York Giants',
    'NYJ': 'New York Jets', 'PHI': 'Philadelphia Eagles', 'PIT': 'Pittsburgh Steelers', 'SF': 'San Francisco 49ers', 'SEA': 'Seattle Seahawks', 'TB': 'Tampa Bay Buccaneers',
    'TEN': 'Tennessee Titans', 'WAS': 'Washington Commanders'
}

## 2. REGULAR EXPRESSIONS

## 3. TRANSFORMING DATA

In [15]:
# TASKS:
# 1. Split 'PlayTimeFormation' feature into 3
#    1. TimeLeftInQuarter
#    2. Quarter (Replace current quarter value, the raw value is not 100% accurate)
#    3. PlayFormation
# 2. Split 'PlayStart' feature into 2
#    1. DownAndDistance
#    2. FieldPosition

In [ ]:
# PURPOSE:
# - Take value for 'PlayTimeFormation' and split this feature into 3
#   separate features.
#   1. TimeLeftInQuarter (Will come about when renaming 'PlayTimeFormation')
#   2. Quarter (This feature already exists, the values within 'PlayTimeFormation' are more accurate and will replace the value in here originaly)
#   3. PlayFormation

def playtimeformation_split(df_plays):

  df_plays_copy = df_plays.copy()

  new_columns = ['PlayFormation']

  df_plays_copy = df_plays_copy.rename(columns = {'PlayTimeFormation': 'TimeLeftInQuarter'})

  df_plays = df_plays.reindex(columns=df_plays.columns.tolist() + new_columns)

## 4. PIPELINE MAIN METHOD

In [16]:
# PURPOSE:
# - Accept a dataframe of nfl plays (formatted by NFL_Scrapers) and
#   return a cleaned dataframe of those plays.
# INPUT PARAMETERS:
# df_all_plays         - dataframe - all plays in raw form from NFL_Scraper that user
#                                    would like to clean.
# OUTPUT:
# df_all_plays_cleaned - dataframe - all plays from 'df_all_plays' cleaned and data
#                                    dispersed into individual new features.

# CURRENT DESIGN PLAN:
# 1. Use uniquely designed methods for each play type to clean within dataframe
#    - (e.g. pass, run, touchdown, punt, sack, ... )
# 2. Repeat until all plays within dataframe have been cleaned.
#   NOTE:
#   - It is important to fully clean a play type before moving to the next
#      because sometimes cleaning could involve adding a new row to the dataframe,
#      causing a reset to the dataframes indexing.
#      - If we were to separate all play types from the beginning, the indexes
#        could shift around causing, for example, an index that might originally
#        point to a run play to now instead point at a pass play.

def clean_dataframe_of_plays(df_all_plays):

  ################################
  # RAW DATA COLUMN DESCRIPTIONS #
  ################################

  # Season             - Year of the season
  # Week               - Game week of the season (e.g. 'Week 1')
  # Day                - Day of the week (e.g. 'MON')
  # Date               - Month and day of the game formatted MM/DD (e.g. '09/07')
  # AwayTeam           - Visiting team of the game
  # HomeTeam           - Home team of the game
  # Quarter            - Quarter that the play is in
  #                      - NOT ACCURATE. Drives that go between quarters will end up
  #                        having all plays in the later quarter.
  # DriveNumber        - Drive number of the quarter that the play is in
  # TeamWithPossession - Team that started with the ball at the beginning of the play.
  # IsScoringDrive     - Does the drive that the focused play in result in a score?
  # PlayNumberInDrive  - Play count in the drive
  # IsScoringPlay      - Did the play result in a score?
  # PlayOutcome        - Ultimate result of the play (e.g. '13 Yard Pass')
  # PlayStart          - The down and where the play started on the field (e.g. '2nd & 9 at DET 21')
  # PlayTimeFormation  - Time left in the quarter / quarter / play formation
  # PlayDescription    - The raw description given of the focused play, entailing everything
  #                      that happened within it.

  ######################################
  # NEW ADDITIONAL COLUMN DESCRIPTIONS #
  ######################################

  # ~ General features ~
  # TimeOnTheClock     - The time that was on the clock when the play started

  # ~ Offensive features ~
  # PlayType           - The type of play (e.g. pass/run)
  # Formation          - Play formation
  # Passer             - Player that threw the ball (mostly the quarterback)
  # Rusher             - Player that ran the ball (mostly the runningback)
  # Receiver           - Player on the same team as the passer that caught the ball
  # Direction          - Where the ball is going during the play
  # Yardage            - Yards gained during the play
  #                      - (Should specify that yardage does not include extra yardage gained from penalties)
  #                      - (Player awarded yardage)
  #                      - (also includes how far kicks have gone during kickoffs and punts)

  # ~ Defensive features ~
  # SoloTackle         - Player awarded a solo tackle from a play
  # AssistedTackle     - Player awarded an assisted tackle from a play
  # SharedTackle       - Player awarded a shared tackle from a play
  # PassDefendedBy     - Defender that defended the passing play
  # PressureBy         - Defender that applied pressure to the passer
  # InterceptedBy      - Defender that intercepted the passing play
  # SackedBy           - Player awarded a sack from a play. (Could be solo or split)
  # ForcedFumbledBy    - Player awarded a forced fumble from a play

  # ~ Unique features (uncommon) ~
  # WhoFumbled         - Player who last held the ball during a fumble.
  # FumbleRecoveredBy  - Player who recovered the fumbled ball
  # FumbleDetails      - A list that has what happened after the fumble
  #                      - [forced fumble by, recovered by, yards gained, tackled by]
  # ReverseDetails     - A list having plays leading up to play reversal
  # InjuredPlayers     - Players that were injured during the play
  # AcceptedPenalty    - Penalty on the field that was accepted
  # DeclinedPenalty    - Penalty on the field that was declined

  # ~ Special teams features ~
  # Kicker             - Player who kicked the ball during a kickoff / punt / extra point / field goal
  # LongSnapper        - Player who snapped the ball during a punt / extra point / field goal
  # Returner           - Player who returned the ball during a kickoff / punt
  # DownedBy           - ? ? ? I forget
  # Holder             - Player who held ball for extra point / field goal
  # BlockedBy          - Player who blocked a punt / extra point / field goal

  new_columns = ["TimeOnTheClock",
                 "PlayType", "Formation", "Passer", "Rusher", "Receiver", "Direction", "Yardage",
                 "SoloTackle", "AssistedTackle", "SharedTackle", 'PassDefendedBy', "PressureBy", "InterceptedBy", "SackedBy", "ForcedFumbleBy",
                 "WhoFumbled", "FumbleRecoveredBy", "FumbleDetails", "ReverseDetails", "InjuredPlayers", "AcceptedPenalty", "DeclinedPenalty",
                 "Kicker", "LongSnapper", "Returner", "DownedBy", "Holder", "BlockedBy"]

  string_columns = ["TimeOnTheClock",
                    "PlayType", "Formation", "Passer", "Rusher", "Receiver", "Direction",
                    "SoloTackle", "AssistedTackle", "SharedTackle", 'PassDefendedBy', "PressureBy", "InterceptedBy", "SackedBy", "ForcedFumbleBy",
                    "WhoFumbled", "FumbleRecoveredBy", "FumbleDetails", "ReverseDetails", "InjuredPlayers", "AcceptedPenalty", "DeclinedPenalty",
                    "Kicker", "LongSnapper", "Returner", "DownedBy", "Holder", "BlockedBy"]

  int_columns = ["Yardage"]

  ########################################
  # RETURN DATAFRAME WITH ADDED FEATURES #
  ########################################

  df_all_plays_cleaned = df_all_plays.copy()
  df_all_plays_cleaned = df_all_plays_cleaned.reindex(columns=df_all_plays_cleaned.columns.tolist() + new_columns)
  df_all_plays_cleaned[string_columns] = df_all_plays_cleaned[string_columns].astype(str)
  df_all_plays_cleaned[int_columns] = df_all_plays_cleaned[int_columns].astype(float)

  #############################################################
  # TRANSFORMING FEATURE VALUES (PREPPING DATA TO BE CLEANED) #
  #############################################################


  ########################################
  # GETTING PLAY CATEGORIES AND CLEANING #
  ########################################



  return df_all_plays_cleaned

# TESTING

In [17]:
df_week2_plays_cleaned = clean_dataframe_of_plays(week2_2023_plays)

In [18]:
df_week2_plays_cleaned.shape

(2713, 45)

# PLAYTYPE OBSERVATIONS

## HELPER METHOD

In [19]:
# PURPOSE:
# - A tool that can be used to compare original plays and their cleaned versions

# I would like to return a map that has:
# KEY: index of original unclean play
# VALUE: index(es) of cleaned play

def unclean_to_clean_play_matches(df_unclean_plays, df_clean_plays):

  my_map = {}

  # This list of features is unique to each play
  # - Both the unclean and cleaned versions of the plays have these same features, therefore
  #   they will be used to match unclean plays in 'df_unclean_plays' to clean plays in 'df_clean_plays'
  matching_features = ['Season', 'Week', 'Date', 'AwayTeam', 'HomeTeam', 'Quarter', 'DriveNumber', 'PlayNumberInDrive']

  # Iterate through each row of the unclean plays dataframe
  for u_row in df_unclean_plays.itertuples(index=True):
    u_features = [getattr(u_row, col) for col in matching_features]

    matching_indexes = []
    matches_found = False

    # Iterate through each row of the dataframe of cleaned plays
    # - The starting index will be the index of the unclean play within the main original dataframe of plays
    #   - The matching cleaned pair will either be at the exact same location or higher
    for c_row in df_clean_plays[u_row.Index::].itertuples(index=True):
      c_features = [getattr(c_row, col) for col in matching_features]

      # If a match is found, check for consective rows of matches because some uncleaned plays needed to be cleaned using multiple rows
      # - Once a row that does not match follows one that does, will break the loop because the one play match has been found.
      if u_features == c_features:
        matching_indexes.append(c_row.Index)
        matches_found = True
      elif matches_found:
        my_map[u_row.Index] = matching_indexes
        break

  return my_map

## PASSING PLAYS

In [ ]:
df_unclean_pass_plays =